In [8]:
!pip install keras_applications
!pip install keras==2.1.5

  Using cached Keras-2.1.5-py2.py3-none-any.whl (334 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.1.5 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import numpy as np


In [10]:
 vgg_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

94694792/94694792 [==============================] - 31s 0us/step


In [11]:
def extract_face_embeddings(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x, version=2)
    features = vgg_model.predict(x)
    return features[0]


In [25]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sqlalchemy import Column, Integer, String, Float, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class FaceEmbedding(Base):
    __tablename__ = 'face_embeddings'

    employee_number = Column(Integer, primary_key=True)
    embeddings = Column(Float, array_dimension=2)
    embedding_type = Column(String)

# Create the database engine and session
engine = create_engine('postgresql://postgres:@localhost:5430/faceio')
Session = sessionmaker(bind=engine)
session = Session()

# Define the directory containing the employee images
image_dir = '/Users/ds_learner16/Documents/Face.io/photos/train'

# Load the VGGFace model
vggface = tf.keras.models.load_model('transfer_learning_trained13_classes_face_cnn_model_data_science.h5')
embedding_model = Model(inputs=vggface.input, outputs=vggface.get_layer('dense_7').output)

# Loop through the directories in the image directory
for dir_name in os.listdir(image_dir):
    if not os.path.isdir(os.path.join(image_dir, dir_name)):
        continue
    
    # Extract the employee number from the directory name
    employee_number = int(dir_name)
    
    # Load and encode all face images for this employee
    embeddings = []
    for file_name in os.listdir(os.path.join(image_dir, dir_name)):
        if not file_name.endswith('.jpg'):
            continue
        
        # Load the image file and extract the face
        image_path = os.path.join(image_dir, dir_name, file_name)
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = tf.keras.applications.vgg16.preprocess_input(x)
        
        # Encode the face using the VGGFace model
        embedding = embedding_model.predict(x)[0]
        embeddings.append(embedding)
    
    # Compute the average face embedding and add it to the database
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        new_embedding = FaceEmbedding(employee_number=employee_number, embeddings=avg_embedding, embedding_type='average')
        new
        session.add(new_embedding)
    
    # Compute the range of face embeddings and add them to the database
    if len(embeddings) > 1:
        min_embedding = np.min(embeddings, axis=0)
        max_embedding = np.max(embeddings, axis=0)
        new_embedding = FaceEmbedding(employee_number=employee_number, embeddings=np.concatenate((min_embedding, max_embedding)), embedding_type='range')
        session.add(new_embedding)
    
session.commit()


/var/folders/7m/m8d42_wx6_s2mq4bkrbnj5_h0000gn/T/ipykernel_14418/2031741598.py:17: SAWarning: Can't validate argument 'array_dimension'; can't locate any SQLAlchemy dialect named 'array'
  embeddings = Column(Float, array_dimension=1)


1/1 [==============================] - 0s 22ms/step


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'numpy.ndarray'
[SQL: INSERT INTO emp_info (employee_number, embeddings, embedding_type) VALUES (%(employee_number)s, %(embeddings)s, %(embedding_type)s)]
[parameters: ({'employee_number': 758, 'embeddings': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'embedding_type': 'average'}, {'employee_number': 758, 'embeddings': array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'embedding_type': 'range'}, {'employee_number': 767, 'embeddings': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32), 'embedding_type': 'average'}, {'employee_number': 767, 'embeddings': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32), 'embedding_type': 'range'}, {'employee_number': 760, 'embeddings': array([0.0000000e+00, 0.0000000e+00

In [19]:
!pip install tensorflow-macos

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.5
    Uninstalling Keras-2.1.5:
      Successfully uninstalled Keras-2.1.5
